# Compare Simulation Runs

Use this notebook to compare results from multiple simulation runs side by side.
Add result folder paths to the `runs` dictionary below and run all cells.

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
from resdata.summary import Summary

# List available results
results_base = '../results/'
result_dirs = sorted(glob.glob(os.path.join(results_base, '*/')))
print('Available results:')
for i, d in enumerate(result_dirs):
    print(f'  [{i}] {os.path.basename(d.rstrip("/"))}')

In [ ]:
# ═══════════════════════════════════════════════════
# DEFINE YOUR RUNS TO COMPARE
# Give each run a label and the path to its results
# ═══════════════════════════════════════════════════
runs = {}

# Automatically load the two most recent runs
if len(result_dirs) >= 2:
    runs['Run A'] = result_dirs[-2]
    runs['Run B'] = result_dirs[-1]
elif len(result_dirs) == 1:
    runs['Run A'] = result_dirs[0]

# Or manually specify:
# runs = {
#     'Base Case': '../results/SPE1CASE1-20260219-100000/',
#     'High Inj':  '../results/SPE1CASE1-20260219-110000/',
# }

# Load summaries
summaries = {}
for label, path in runs.items():
    smspec = glob.glob(os.path.join(path, '*.SMSPEC'))
    if smspec:
        summaries[label] = Summary(smspec[0])
        print(f'Loaded {label}: {smspec[0]}')
    else:
        print(f'WARNING: No .SMSPEC found for {label} in {path}')

print(f'\nComparing {len(summaries)} runs')

In [ ]:
# ═══════════════════════════════════════════════════
# Compare Field Totals
# ═══════════════════════════════════════════════════
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Field Comparison', fontsize=16, fontweight='bold')

plots = [
    ('FOPT', 'Oil Production Total', 'STB'),
    ('FGPT', 'Gas Production Total', 'MSCF'),
    ('FWPT', 'Water Production Total', 'STB'),
    ('FPR',  'Field Pressure', 'PSIA'),
]

colors = ['#2563eb', '#dc2626', '#059669', '#d97706', '#7c3aed']

for ax, (key, title, unit) in zip(axes.flat, plots):
    for i, (label, s) in enumerate(summaries.items()):
        try:
            values = s.numpy_vector(key)
            ax.plot(s.dates, values, linewidth=2, label=label, color=colors[i % len(colors)])
        except KeyError:
            pass
    ax.set_title(title)
    ax.set_ylabel(unit)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════
# Compare Field Rates
# ═══════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.suptitle('Rate Comparison', fontsize=16, fontweight='bold')

rate_plots = [
    ('FOPR', 'Oil Rate', 'STB/day'),
    ('FGPR', 'Gas Rate', 'MSCF/day'),
    ('FWPR', 'Water Rate', 'STB/day'),
]

for ax, (key, title, unit) in zip(axes.flat, rate_plots):
    for i, (label, s) in enumerate(summaries.items()):
        try:
            values = s.numpy_vector(key)
            ax.plot(s.dates, values, linewidth=2, label=label, color=colors[i % len(colors)])
        except KeyError:
            pass
    ax.set_title(title)
    ax.set_ylabel(unit)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════
# Summary Table — Final Values
# ═══════════════════════════════════════════════════
import pandas as pd

keys_to_compare = ['FOPT', 'FGPT', 'FWPT', 'FPR', 'FOPR', 'FGPR', 'FWPR']
rows = []

for label, s in summaries.items():
    row = {'Run': label}
    for key in keys_to_compare:
        try:
            row[key] = f'{s.numpy_vector(key)[-1]:,.1f}'
        except KeyError:
            row[key] = 'N/A'
    rows.append(row)

df = pd.DataFrame(rows).set_index('Run')
print('Final timestep values:')
print()
df